In [1]:
import sys
import torch
import numpy as np
import pandas as pd

In [2]:
from gibbsibp import UncollapsedGibbsIBP

In [3]:
dataset = pd.read_excel('training_data.xlsx')
dataset.head()

,cp_pres_1,cp_pres_2,cp_pres_3,cp_pres_4,cp_pres_5,cp_pres_6,cp_pres_7,cp_pres_8,cp_pres_9,cp_pres_10,...,cp_pres_14,cp_pres_15,cp_pres_16,cp_pres_17,cp_pres_18,cp_pres_19,cp_pres_20,cp_pres_21,cp_pres_22,cp_pres_23
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:

conditions = []
                                                                                                # COMMUNICATION FROM SABYA:
conditions.append({'force':np.array([ 0., 0.]),             'cue':np.array([0., 0., 0., 0.,])}) # 0: is null/ control you can ignore
conditions.append({'force':np.array([-0.866025404, 0.5]),   'cue':np.array([1., 0., 0., 0.,])}) # 1,2: elements
conditions.append({'force':np.array([ 0.866025404, 0.5]),   'cue':np.array([0., 0., 1., 0.,])})
conditions.append({'force':np.array([ 0.,-1.]),             'cue':np.array([0., 1., 0., 0.,])}) # 3: not visual and not dynamic compositional
conditions.append({'force':np.array([ 0.866025404, -0.5]),  'cue':np.array([0., 0., 0., 1.,])}) # 4: not visual but dynamic compositional
conditions.append({'force':np.array([ 0, 1]),               'cue':np.array([1., 0., 1., 0.,])}) # 5: visual and dynamic compositional
conditions.append({'force':np.array([-0.866025404, -0.5]),  'cue':np.array([0., 1., 1., 0.,])}) # 6: not dynamic but visual compositional

for condition in conditions:
    condition['force'] = condition['force']/4

In [5]:
def generate_input_data(condition_seq):
    """
    Generate input data for the model, based on sequences of conditions experienced by the subjects.
    """
    X_dataset = []
    F_dataset = []

    for condition in condition_seq:
        X_dataset.append(conditions[condition]['cue'])
        F_dataset.append(conditions[condition]['force'])

    X_dataset = torch.tensor(X_dataset, dtype=torch.float32)
    F_dataset = torch.tensor(F_dataset, dtype=torch.float32)

    return X_dataset, F_dataset

In [17]:
seq_1 = dataset['cp_pres_1'].to_list()
seq_1 = [int(x) for x in seq_1]
seq_1 = [x for x in seq_1 if x != 0]
print(len(seq_1))
#seq_1 = seq_1[:200]

X_dataset, F_dataset = generate_input_data(seq_1)

630


In [7]:
# def add_noise_to_obs(X, F, F_noise_std = 0.01, lambd=0.98, epsilon=0.02):
#     """
#     Add gaussian noise to force data, and randomly flip pixels in the observation data
#     """
#     F += torch.randn(F.size()) * F_noise_std
    
#     X_noisy = torch.zeros(X.size())
#     for i in range(X.size()[0]):
#         for j in range(X.size()[1]):
#             if X[i, j] == 1:
#                 X_noisy[i, j] = 1 if random() < lambd else 0
#             else:
#                 X_noisy[i, j] = 1 if random() < epsilon else 0

#     return X_noisy, F

In [8]:
# X_dataset, F_dataset = add_noise_to_obs(X_dataset, F_dataset)

print(X_dataset[:5,])
print(F_dataset[:5,])

tensor([[0., 1., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.]])
tensor([[-0.2165, -0.1250],
        [-0.2165,  0.1250],
        [ 0.2165, -0.1250],
        [ 0.2165,  0.1250],
        [ 0.0000, -0.2500]])


In [9]:
import cProfile


In [18]:
inf = UncollapsedGibbsIBP(alpha=0.05, K=1, max_K=4, sigma_a=0.2, sigma_n=0.1, epsilon=0.05, lambd=0.99, phi=0.25)
with cProfile.Profile() as pr:
    As, Zs, Ys = inf.gibbs(F_dataset, X_dataset, 5)

pr.print_stats(sort='cumtime')

         4115206 function calls (4108906 primitive calls) in 15.205 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   15.205   15.205 gibbsibp.py:536(gibbs)
        5    0.095    0.019   15.138    3.028 gibbsibp.py:380(resample_Z)
     3150    1.094    0.000   14.016    0.004 gibbsibp.py:330(sample_k_new)
    31500    2.499    0.000    5.190    0.000 gibbsibp.py:245(F_loglik_given_k_new)
    31500    1.679    0.000    4.621    0.000 gibbsibp.py:291(X_loglik_given_k_new)
    31500    0.368    0.000    1.919    0.000 poisson.py:66(log_prob)
   370638    0.220    0.000    1.649    0.000 _tensor.py:34(wrapped)
   237030    0.106    0.000    1.010    0.000 _tensor.py:907(__rsub__)
    28362    1.007    0.000    1.007    0.000 {method 'inverse' of 'torch._C._TensorBase' objects}
37804/31504    0.040    0.000    0.997    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
   

In [19]:
inf = UncollapsedGibbsIBP(alpha=0.05, K=1, max_K=6, sigma_a=0.2, sigma_n=0.05, epsilon=0.01, lambd=0.99, phi=0.25)

As, Zs, Ys = inf.gibbs(F_dataset, X_dataset, 100)

In [27]:
A, Z, Y = extract_mean_from_samples(As, Zs, Ys, n=95)
print(A*4)
print(Y)
print(Z)

[[ 0.88 -0.48]
 [ 0.88  0.52]
 [-0.88 -0.52]
 [ 0.   -1.  ]
 [-0.88  0.52]]
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
[[0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]]


In [21]:
def extract_mean_from_samples(As, Zs, Ys, n=10):
    A_mean = np.round(np.mean(np.array(As[-n:]),axis=0), 2)
    Z_mean = np.round(np.mean(np.array(Zs[-n:]),axis=0), 2)
    Y_mean = np.round(np.mean(np.array(Ys[-n:]),axis=0), 2)

    return A_mean, Z_mean, Y_mean

def compare_distance(reference_matrix, inferred_matrix):
    """
    Compare the distance between rows of the reference and the true matrix.
    use this to create a permutation matrix that reorders the inffered matrix to match the reference matrix,
    and return the permutation matrix
    """
    n, m = reference_matrix.shape
    assert inferred_matrix.shape == (n, m)

    # compute the distance matrix
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            distance_matrix[i, j] = np.linalg.norm(reference_matrix[i] - inferred_matrix[j])

    # find the permutation that minimizes the distance
    from scipy.optimize import linear_sum_assignment
    row_ind, col_ind = linear_sum_assignment(distance_matrix)
    
    # create the permutation matrix that corresponds to this reordering
    permutation_matrix = np.zeros((n, n))
    for i in range(n):
        permutation_matrix[i, col_ind[i]] = 1

    return permutation_matrix


In [ ]:
A, Z, Y = extract_mean_from_samples(As, Zs, Ys, n=10)

reorder = compare_distance(true_A.numpy(), A)

print("True A:")
print(true_A.numpy())
print("Inferred A:")
print(np.round(reorder @ A,2))

print("\nTrue Y:")
print(true_Y.numpy())
print("Inferred Y:")
print(np.round(reorder @ Y,2))

print("\nTrue Z:")
print(np.array(Z_latent))
print("Inferred Z:")
print(np.round(Z @ reorder,0))

True A:
[[ 0.2  0.2]
 [-0.2  0.2]
 [ 0.  -0.2]]
Inferred A:
[[ 0.21  0.19]
 [-0.2   0.18]
 [-0.01 -0.17]]

True Y:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]
Inferred Y:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]

True Z:
[[1 0 0]
 [0 1 0]
 [0 0 1]
 [1 1 0]
 [1 0 1]
 [0 1 1]
 [1 1 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 1 0]
 [1 0 1]
 [0 1 1]
 [1 1 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 1 0]
 [1 0 1]
 [0 1 1]
 [1 1 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 1 0]
 [1 0 1]
 [0 1 1]
 [1 1 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 1 0]
 [1 0 1]
 [0 1 1]
 [1 1 1]]
Inferred Z:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 1. 0.]
 [1. 0. 1.]
 [0. 1. 1.]
 [1. 1. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 1. 0.]
 [1. 0. 1.]
 [0. 1. 1.]
 [1. 1. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 1. 0.]
 [1. 0. 1.]
 [0. 1. 1.]
 [1. 1. 1.]
 [1. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 1. 0.]
 [1. 0. 1.]
 [0. 1. 1.]
 [1. 1. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 1. 0.]
 [1. 0. 1.]
 [0. 1. 1.]
 [1. 1. 1.]]
